# Jointure MapReduce

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader
import org.apache.spark.sql.functions._
import java.sql.Date

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experiment: ExperimentConfig = configuration.experiments.head

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@530e1568
session = org.apache.spark.sql.SparkSession@530e1568
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputC...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputC...

# Chargement Flight et Weather

In [13]:
// Chargement de Flight Data
val processedFlightPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val flightDF = spark.read.parquet(processedFlightPath)

val processedWeatherPath = s"${configuration.common.output.basePath}/common/data/processed_weather.parquet"
val weatherDF = spark.read.parquet(processedWeatherPath)



processedFlightPath = /home/jovyan/work/output/common/data/processed_flights.parquet
flightDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]
processedWeatherPath = /home/jovyan/work/output/common/data/processed_weather.parquet
weatherDF = [WBAN: string, Date: date ... 65 more fields]


[WBAN: string, Date: date ... 65 more fields]

# Filtres

In [15]:
import java.sql.Date

val weatherData = weatherDF
/**  .filter(
    col("Date") === Date.valueOf("2012-01-09") &&
    (col("WBAN") === "13869" || col("WBAN") === "13874")
  )
  .orderBy("WBAN", "Date", "Time")
**/
val flightData = flightDF
/**  .filter(
    col("UTC_FL_DATE") === "2012-01-09" &&
    col("OP_CARRIER_AIRLINE_ID") === 20366 &&
    col("OP_CARRIER_FL_NUM") === 5334 && 
    col("ORIGIN_WBAN") === 13869  
  )**/


weatherData = [WBAN: string, Date: date ... 65 more fields]
flightData = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 109 more fields]

# Feature Pipeline

In [16]:
import com.flightdelay.features.FeaturePipeline

val joinedData = FeaturePipeline.join(flightData, weatherData, experiment)
joinedData.show(1, 1000, true)


Joining flight and weather data...
  - Flight records: 134,556
  - Weather records: 1,414,384
[FlightWeatherJoinner] Colonnes ajoutées automatiquement pour origin: feature_utc_departure_hour_rounded, UTC_ARR_DATE, UTC_FL_DATE, feature_utc_arrival_hour_rounded
[FlightWeatherJoinner] Colonnes ajoutées automatiquement pour destination: feature_utc_departure_hour_rounded, UTC_ARR_DATE, UTC_FL_DATE, feature_utc_arrival_hour_rounded
[Anti-Leakage] Colonnes supprimées : UTC_ARR_DATE, feature_utc_arrival_hour_rounded, feature_utc_departure_hour_rounded
[Anti-Leakage] Time transformé en hours_before_flight (index relatif)
[Anti-Leakage] Champs conservés dans observations météo : SkyCondition, Visibility, hours_before_flight
  - Joined records: 204,386 with  11 columns

Saving joined flight weather data to parquet:
  - Path: /home/jovyan/work/output/Experience-Juniper-1/data/joined_flights_weather.parquet
  - Saved 204386 joined records
-RECORD 0-------------------------------------------------

joinedData = [DEST_AIRPORT_ID: int, DEST_WBAN: string ... 9 more fields]


[DEST_AIRPORT_ID: int, DEST_WBAN: string ... 9 more fields]

In [17]:
import com.flightdelay.features.FeaturePipeline

val explosedData = FeaturePipeline.explose(joinedData, experiment)
explosedData.show(1, 1000, true)


Exploding weather observation arrays:
  - Weather features: SkyCondition, Visibility
  - Depth hours: 12 observations
  - Input columns: 11
  - Exploded origin_weather_observations into 24 columns
  - Exploded destination_weather_observations into 24 columns
  - Total added columns: 48
  - Output columns: 57
  - Column organization: grouped by index (11 to 0)

Saving exploded data to parquet:
  - Path: /home/jovyan/work/output/Experience-Juniper-1/data/exploded_joined_data.parquet
  - Saved 204386 exploded records
-RECORD 0-----------------------------------------
 DEST_AIRPORT_ID                     | 10140      
 DEST_WBAN                           | 23050      
 FL_DATE                             | 2012-01-01 
 OP_CARRIER_AIRLINE_ID               | 20398      
 OP_CARRIER_FL_NUM                   | 3864       
 ORIGIN_AIRPORT_ID                   | 13930      
 ORIGIN_WBAN                         | 94846      
 UTC_FL_DATE                         | 2012-01-02 
 label_is_delayed_15

explosedData = [DEST_AIRPORT_ID: int, DEST_WBAN: string ... 55 more fields]


[DEST_AIRPORT_ID: int, DEST_WBAN: string ... 55 more fields]

In [18]:
import com.flightdelay.features.FeaturePipeline

val extractedFeatureData = FeaturePipeline.extractFeature(explosedData, experiment)



[DataLeakageProtection] Cleaning

[LeakageProtection] Removing Source Leakage Columns
  ✓ No source leakage columns found - Data is clean!

[LeakageProtection] Removing Unused Labels
  Target label: label_is_delayed_15min
  ✓ No unused labels found
  - Data leakage protection completed in 0.0s

  - Checking cardinality for 4 integer columns (sample: 1.0%)
  - Reclassified 1 low-cardinality integers as categorical: OP_CARRIER_AIRLINE_ID
  - Column type detection completed in 10.492s
  - Detected 27 categorical columns and 27 numeric columns

  - Using EnhancedFlightFeaturePipeline (Feature Selection mode)
  - Feature pipeline completed in 33.781s
  - Feature count: 54 features
  - Sample feature names: indexed_DEST_WBAN, indexed_ORIGIN_WBAN, indexed_origin_weather_SkyCondition-11, indexed_origin_weather_SkyCondition-10, indexed_origin_weather_SkyCondition-9...

Sample of extracted features:
+-----------------------------------------------------------------------------------------------

extractedFeatureData = [features: vector, label: double]


[features: vector, label: double]

In [9]:
extractedFeatureData.show(2, 1000)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|                                                                                                                                                                                                    features|label|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(54,[26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])|  0.0|
+---------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
explosedData.show(10,1000, true)

-RECORD 0------------------------------------------------------
 DEST_AIRPORT_ID                     | 10397                   
 DEST_WBAN                           | 13874                   
 FL_DATE                             | 2012-01-09              
 OP_CARRIER_AIRLINE_ID               | 20366                   
 OP_CARRIER_FL_NUM                   | 5334                    
 ORIGIN_AIRPORT_ID                   | 10146                   
 ORIGIN_WBAN                         | 13869                   
 UTC_FL_DATE                         | 2012-01-09              
 label_is_delayed_15min              | 1                       
 origin_weather_SkyCondition-11      | OVC100                  
 origin_weather_Visibility-11        | 1.75                    
 origin_weather_SkyCondition-10      | OVC110                  
 origin_weather_Visibility-10        | 9.0                     
 origin_weather_SkyCondition-9       | OVC110                  
 origin_weather_Visibility-9         | 1

In [10]:
import com.flightdelay.features.leakage.DataLeakageProtection
val cleaData = DataLeakageProtection.clean(explosedData, "label_is_delayed_15min")


[DataLeakageProtection] Cleaning

[LeakageProtection] Removing Source Leakage Columns
  ✓ No source leakage columns found - Data is clean!

[LeakageProtection] Removing Unused Labels
  Target label: label_is_delayed_15min
  ✓ No unused labels found


cleaData = [DEST_AIRPORT_ID: int, DEST_WBAN: string ... 55 more fields]


[DEST_AIRPORT_ID: int, DEST_WBAN: string ... 55 more fields]

In [11]:
import com.flightdelay.data.utils.DataQualityMetrics
val dataMetric = DataQualityMetrics.metrics(cleaData)

dataMetric = [name: string, origType: string ... 3 more fields]


[name: string, origType: string ... 3 more fields]

In [12]:
dataMetric.show(100)

+--------------------+-----------+-------+---------+----------------+
|                name|   origType|colType|compRatio|nbDistinctValues|
+--------------------+-----------+-------+---------+----------------+
|     DEST_AIRPORT_ID|IntegerType|numeric|      1.0|               1|
|           DEST_WBAN| StringType|textual|      1.0|               1|
|             FL_DATE|   DateType|   date|      1.0|               1|
|OP_CARRIER_AIRLIN...|IntegerType|numeric|      1.0|               1|
|   OP_CARRIER_FL_NUM|IntegerType|numeric|      1.0|               1|
|   ORIGIN_AIRPORT_ID|IntegerType|numeric|      1.0|               1|
|         ORIGIN_WBAN| StringType|textual|      1.0|               1|
|         UTC_FL_DATE|   DateType|   date|      1.0|               1|
|label_is_delayed_...|IntegerType|numeric|      1.0|               1|
|origin_weather_Sk...| StringType|textual|      1.0|               1|
|origin_weather_Vi...| DoubleType|numeric|      1.0|               1|
|origin_weather_Sk..